#Amazon DataSet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/Shareddrives/IDS Project/'

In [ ]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler  # For scaling dataset
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline

## Data Prepocessing



### Amazon CSV


In [ ]:
df = pd.read_csv(path + 'All_Beauty.csv', header = None)
df.columns = ["Item", "User", "Rating", "Timestamp"]
df

In [ ]:
import datetime as dt

timestamp = df["Timestamp"]
dateTime = []

for ts in timestamp:
    ab = dt.datetime.fromtimestamp(int(ts)).date()
    dateTime.insert(ts, ab)

df['Datetime'] = pd.Series(dateTime)
df.drop_duplicates(inplace=True)
df

### Amazon JSON

In [ ]:
entries = []
for line in open(path + 'meta_All_Beauty.json', 'r'):
    entries.append(json.loads(line))

In [ ]:
df2 = pd.DataFrame(entries)
df2.columns

In [ ]:
df2 = pd.DataFrame(entries)
df2

In [ ]:
df2 = pd.DataFrame(entries)
df2 = df2.replace("", np.nan)
df2.drop(columns=['category', 'tech1', 'fit', 'tech2', 'feature'], inplace=True)
df2.description = df2.description.apply(lambda y: np.nan if len(y)==0 else y)
df2.also_buy = df2.also_buy.apply(lambda y: np.nan if len(y)==0 else y)
df2.image = df2.image.apply(lambda y: np.nan if len(y)==0 else y)
df2.also_view = df2.also_view.apply(lambda y: np.nan if len(y)==0 else y)
df2.details = df2.details.apply(lambda y: np.nan if len(y)==0 else y)
df2['rank'] = df2['rank'].apply(lambda y: np.nan if len(y)==0 else y)
df2['price'] = df2['price'].replace({'\$':''}, regex = True)
df2['price']  = pd.to_numeric(df2['price'] , errors='coerce')
bpr = (df2['rank'].str.contains('Beauty')) & (df2['asin'].notnull())
df2['rank'] = df2['rank'][bpr].str.extract('(.*) in', expand = False)
df2['rank'] = df2['rank'].str.replace(",","")
df2['rank'] = pd.to_numeric(df2['rank'], errors='coerce')
df2.to_csv('BeautyJson.csv')
df2

## Research Question

### Q1 Descriptive Question
Which month or period has the highest sales for the Amazon beauty products?

In [ ]:
df.sort_values('Datetime', ascending = True) # Datetime shows from 2000 - 2018
df['Datetime'].nunique()  # We get 5288 unique date rows here 

In [ ]:
#Dataframe that shows number of sales of every item each day
salescount = df.groupby(['Datetime', 'Item'])['Item'].count().rename("Sales")
salescount = salescount.to_frame()
salescount

In [ ]:
#Dataframe that shows the sum up number of sales in each day
sumup = salescount.groupby(['Datetime'])['Sales'].sum().reset_index(name = "Sum of item")
sumup

In [ ]:
#Add a new column "Month" which extract from Datetime 
sumup['Datetime'] = sumup['Datetime'].astype(str)
sumup['Month'] = sumup['Datetime'].str.split('-').str[1]
sumup

In [ ]:
#Dataframe that shows the total items accumulated in every month 
summonth = sumup.groupby(['Month'])['Sum of item'].sum().reset_index(name = "Total items accumulated in particular month")
summonth

In [ ]:
#Numerical style month changed to word style month
look_up = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
summonth['Month'] = summonth['Month'].apply(lambda x: look_up[x])
summonth["Total items accumulated in particular month"].sum()   #362275
summonth

In [ ]:
summonth.describe()

Extra dataframe that shows total items accumulated in every year.

In [ ]:
salescount2 = df.groupby(['Datetime', 'Item'])['Item'].count().rename("Sales")
salescount2 = salescount2.to_frame()
salescount2

sumup2 = salescount2.groupby(['Datetime'])['Sales'].sum().reset_index(name = "Sum of item")
sumup2

sumup2['Datetime'] = sumup2['Datetime'].astype(str)
sumup2['Year'] = sumup2['Datetime'].str.split('-').str[0]
sumup2

sumyear = sumup2.groupby(['Year'])['Sum of item'].sum().reset_index(name = "Total items accumulated in that particular year")
sumyear["Total items accumulated in that particular year"].sum()   #362275
sumyear

#### Data Visualisation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
  
summonth.plot(x = 'Month', y = 'Total items accumulated in particular month', kind = 'line', figsize = (15,8))
plt.ylim(25000, 35000)

plt.title('Total number of beauty products sold in every particular month from 2000 - 2018')
plt.xticks(summonth.index, summonth['Month'].values)
plt.ylabel('Total Items Count')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
  
sumyear.plot(x = 'Year', y = 'Total items accumulated in that particular year', kind = 'line', figsize = (15,8))
plt.ylim(0, 110000)

plt.title('Total number of beauty products sold in every particular year from 2000 - 2018')
plt.xticks(sumyear.index, sumyear['Year'].values)
plt.ylabel('Total Items Count')
plt.show()

### Q2 Causal Question
Does higher rating of beuaty products leads to higher amount of sale? 

In [ ]:
rate = df.groupby(['Item'])["Rating"].mean().sort_index()
rate.to_frame()

In [ ]:
sale = df['Item'].value_counts().rename(index = 'Sale').sort_index()
sale.to_frame()

In [ ]:
rate_sale = pd.merge(rate,sale,on=sale.index).rename(columns={'key_0':'Item'})
rate_sale = rate_sale.sort_values(by=['Rating'],ascending=False)
rate_sale

In [ ]:
rate_sale.describe()

#### Data Visualisation

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.regplot(x='Rating', y='Sale', data=rate_sale, scatter_kws={"color": "blue"}, line_kws={"color": "red"})

In [ ]:
r = rate_sale[['Rating','Sale']].corr()
print("Correlation Coefficient, r")
print(r,"\n")

sns.heatmap(r, cmap='rainbow')

In [ ]:
rSquare = r * r
print("Coefficient of Determination, r^2")
print(rSquare)

### Q3 Exploratory Question
What type of product is most frequently bought by people who buy  Amazon beauty products?


In [ ]:
#DataFrame Item and Sale
sale1 = df['Item'].value_counts().rename(index = 'Sale').sort_index()
dfSale = sale1.to_frame().reset_index()
dfSale = dfSale.rename(columns={'index': 'Item'})
dfSale

In [ ]:
df3 = pd.DataFrame(data=df2, columns=['asin','also_buy', 'price', 'rank'])
df3 = df3.rename(columns={'asin':'Item'})
df3.dropna(subset = ['also_buy'], inplace=True)
df3

In [ ]:
#Calculate the appearance number of also_buy
numBuy = []
for x in range(df3['also_buy'].str.len().max()):
    buyList = df3['also_buy'].str.get(x)
    for y in buyList:
        numBuy.append(y)
numBuy2 = pd.DataFrame(data=numBuy)
numBuy2 = numBuy2.value_counts()
df4 = pd.DataFrame(data=numBuy2, columns=['Number of Also Buy']).reset_index()
df4.rename(columns={0: 'Item'}, inplace=True)
df4

In [ ]:
df5 = pd.merge(df3, df4, on=["Item"])
df5

In [ ]:
dfAlsoBuy = pd.merge(dfSale, df5, on=["Item"])
dfAlsoBuy.drop(columns=['also_buy'], inplace=True)
dfAlsoBuy.drop_duplicates(inplace=True)
dfAlsoBuy.dropna(subset = ["price"], inplace=True)
dfAlsoBuy.reset_index(inplace=True)
dfAlsoBuy.drop(columns=['index'], inplace=True)
dfAlsoBuy

In [ ]:
df6 = pd.DataFrame(data=df2, columns=['asin','also_view', 'price', 'rank'])
df6 = df6.rename(columns={'asin':'Item'})
df6.dropna(subset = ['also_view'], inplace=True)
df6

In [ ]:
numView = []
for x in range(df6['also_view'].str.len().max()):
    viewList = df6['also_view'].str.get(x)
    for y in viewList:
        numView.append(y)
numView2 = pd.DataFrame(data=numView)
numView2 = numView2.value_counts()
df7 = pd.DataFrame(data=numView2, columns=['Number of Also View']).reset_index()
df7.rename(columns={0: 'Item'}, inplace=True)
df7

In [ ]:
df8 = pd.merge(df6, df7, on=["Item"])
df8

In [ ]:
dfAlsoView = pd.merge(dfSale, df8, on=["Item"])
dfAlsoView.drop(columns=['also_view'], inplace=True)
dfAlsoView.drop_duplicates(inplace=True)
dfAlsoView.dropna(subset=['price', 'rank'], inplace=True)
dfAlsoView.reset_index(inplace=True)
dfAlsoView.drop(columns=['index'], inplace=True)
dfAlsoView

####Correlational Analysis

In [ ]:
dfAlsoBuyView = pd.merge(dfAlsoBuy, dfAlsoView, on=["Item", 'Sale', 'price', 'rank'])
dfAlsoBuyView

In [ ]:
cor = dfAlsoBuyView.corr()
display(cor) 

In [ ]:
sns.heatmap(cor, square = True)

####Data Mining: Clustering

In [ ]:
dfAlsoBuyView

In [ ]:
filter = dfAlsoBuyView.groupby(['Sale'])[['Number of Also Buy', 'Number of Also View', 'price', 'rank']].mean()
filter

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(filter)
print(filter)
print(X) 

In [ ]:
dfClus = pd.DataFrame(data=filter)
dfClus

In [ ]:
def plot_kmeans_scatter2(v1,v2,kmeans_labels):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    scatter = ax.scatter(v1,v2,c=kmeans_labels[0],s=50,cmap='jet',alpha=0.7)
    ax.set_title('K-Means Clustering')
    ax.set_xlabel(v1.name)
    ax.set_ylabel(v2.name)
    plt.colorbar(scatter)
    plt.show()

In [ ]:
v1 = dfClus['Number of Also Buy']
v2 = dfClus['Number of Also View']
v3 = dfClus['price']
v4 = dfClus['rank']

In [ ]:
print("K Mean = 2")
model= KMeans(2,verbose=0)
model.fit(X)
print(model.labels_.shape)
print(model.labels_)
print("Inertia (kmean = 2):", model.inertia_)
print("Iteration (kmean = 2):", model.n_iter_)
kmeans_labels = pd.DataFrame(model.labels_)
print("\n")

In [ ]:
plot_kmeans_scatter2(v2,v1,kmeans_labels) 
plot_kmeans_scatter2(v4,v1,kmeans_labels) 
plot_kmeans_scatter2(v4,v2,kmeans_labels) 

###Q4 Predictive Question
Will people buy the product which has the highest ranking in beauty and personal care?

In [ ]:
dfSale

In [ ]:
df4

####Linear Regression

In [ ]:
dfRank = pd.DataFrame(data=df2, columns=['asin','rank'])
dfRank = dfRank.rename(columns={'asin':'Item'})
dfRank.dropna(subset=['rank'], inplace=True)
dfRank

In [ ]:
dfmodel = pd.merge(dfRank, dfSale, on=["Item"])
dfmodel

In [ ]:
dfmodel1 = pd.merge(df4, dfmodel, on=["Item"])
dfmodel1

In [ ]:
dfmodel2 = pd.DataFrame(data=dfmodel1)
dfmodel2.drop(columns=['Item'], inplace=True)
dfmodel2

In [ ]:
dfmodel2.describe()

In [ ]:
X = dfmodel2.drop("Sale", axis=1)     # drop labels from original data
y = dfmodel2["Sale"].copy() 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
reg = LinearRegression().fit(X_train, y_train) 

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

In [ ]:
reg.score(X_test, y_test)

In [ ]:
some_data = X_test.iloc[:5]   # take first 5 entries from X_test
predicted_sales_values = reg.predict(some_data)
predicted_sales_values

In [ ]:
actual_sales_values = y_test.iloc[:5].values     # here's the first 5 entries from y_test, which are the 'ground truth'
actual_sales_values

In [ ]:
np.abs(predicted_sales_values-actual_sales_values)

In [ ]:
sns.regplot(x="rank", y="Sale", data=dfmodel2, 
            scatter_kws={"color": "blue"}, line_kws={"color": "red"})
plt.show()